---
### Universidad de Costa Rica
#### IE0405 - Modelos Probabilísticos de Señales y Sistemas
---

# `Py5` - *Curvas de ajuste de datos*

> Los modelos para describir un fenómeno y sus parámetros pueden obtenerse a partir de una muestra de datos. Debido a la gran cantidad de modelos probabilísticos disponibles, a menudo es necesario hacer una comparación de ajuste entre muchas de ellas.

---

## Ajuste de modelos

El ajuste de modelos...

### Estimación de máxima verosimilitud (MLE)

(Esto es de menor prioridad) La estimación de máxima verosimilitud (**MLE**, *maximum likelihood estimation*) es...


---
## 5.1 - Con el módulo `stats`

Para iniciar ...

---
## 5.2 - Con la librería `fitter`

Si es necesario ...

---
### Más información

* [Página web](https://www.google.com/)
* Libro o algo
* Tutorial [w3schools](https://www.w3schools.com/python/)
---

---

**Universidad de Costa Rica**

Facultad de Ingeniería

Escuela de Ingeniería Eléctrica

---